# User Guide: Processing data using OptionVolume

## OptionVolume_20190221

### _New Features and Changes_
- Designed based on data file OptVol_20190221.xlsx
    - Option Volume data range: 2015-01-01 to 2019-02-21, 5D rule
    - Option Ticker requestign datepoint: 2016-01-08 to 2019-02-21, length = 39 
    
    
- __get_volume method:__
    - allow requesting cummulative option volume for 'call' 'put' separately, by setting call_put = 'A'
    - allow automatically store requested data in excel, by setting export2excel = True, and excelname = 'Whatever name you want.xlsx'


- Option Ticker delimiter: " " -> "_"
- Expiration data name rule: mm/dd/yy -> yyyymmdd

### _Incoming features_
- embed weekly/monthly expiration periodicity

In [1]:
import pandas as pd
from OptionVolume_20190221 import OptionVolume # import OptionVolume in file OptionVolume_20190215.py
file_location = '../data/OptVol_20190221.xlsx'
workbook = pd.ExcelFile(file_location)

In [2]:
obj = OptionVolume(workbook,'AAPL')
df1 = obj.get_volume(expdates=['20190215','20190222']) # return daily volume that aggregated on all call/put strikes for 2 expiration dates
df2 = obj.get_volume(expdates=['20190215','20190222'],call_put='Call') # return call option daily volume that aggregated on all strikes for 2 expiration dates
df3 = obj.get_volume(expdates='20190222',call_put='P') # return put option daily volume that aggregated on all strikes for 02/22/19
df4 = obj.get_volume(expdates='20190222',call_put='A') # return call put option volume separately
df5 = obj.get_volume(call_put='A') # return put option daily volume that aggregated on all strikes for all existed expiration dates
# df_ = obj.get_volume(call_put='A',export2excel=True,excelname='tmp_excelname.xlsx') # export user result to excel

In [3]:
obj.expdates_existed # all expiration dates existed in data

['20160108',
 '20160115',
 '20160122',
 '20160129',
 '20160205',
 '20160212',
 '20160219',
 '20160226',
 '20160304',
 '20160311',
 '20160318',
 '20160324',
 '20160401',
 '20160408',
 '20160415',
 '20160422',
 '20160429',
 '20160506',
 '20160513',
 '20160520',
 '20160527',
 '20160603',
 '20160610',
 '20160617',
 '20160624',
 '20160701',
 '20160708',
 '20160715',
 '20160722',
 '20160729',
 '20160805',
 '20160812',
 '20160819',
 '20160826',
 '20160902',
 '20160909',
 '20160916',
 '20160923',
 '20160930',
 '20161007',
 '20161014',
 '20161021',
 '20161028',
 '20161104',
 '20161111',
 '20161118',
 '20161125',
 '20161202',
 '20161209',
 '20161216',
 '20161223',
 '20161230',
 '20170106',
 '20170113',
 '20170120',
 '20170127',
 '20170203',
 '20170210',
 '20170217',
 '20170224',
 '20170303',
 '20170310',
 '20170317',
 '20170324',
 '20170331',
 '20170407',
 '20170413',
 '20170421',
 '20170428',
 '20170505',
 '20170512',
 '20170519',
 '20170526',
 '20170602',
 '20170609',
 '20170616',
 '20170623',

In [4]:
df1.iloc[-7:]

,AAPL_20190215,AAPL_20190222
2019-02-13,189833,41511
2019-02-14,176824,53514
2019-02-15,190436,83591
2019-02-18,0,0
2019-02-19,0,118929
2019-02-20,0,183276
2019-02-21,0,136927


In [5]:
df2.iloc[-7:]

,AAPL_20190215_Call,AAPL_20190222_Call
2019-02-13,87356,27858
2019-02-14,108278,34046
2019-02-15,109341,51022
2019-02-18,0,0
2019-02-19,0,75412
2019-02-20,0,127469
2019-02-21,0,88900


In [6]:
df3.iloc[-7:]

,AAPL_20190222_P
2019-02-13,13653
2019-02-14,19468
2019-02-15,32569
2019-02-18,0
2019-02-19,43517
2019-02-20,55807
2019-02-21,48027


In [7]:
df4.iloc[-7:]

,AAPL_20190222_Call,AAPL_20190222_Put
2019-02-13,27858,13653
2019-02-14,34046,19468
2019-02-15,51022,32569
2019-02-18,0,0
2019-02-19,75412,43517
2019-02-20,127469,55807
2019-02-21,88900,48027


In [8]:
df5.iloc[-7:]

,AAPL_20160108_Call,AAPL_20160108_Put,AAPL_20160115_Call,AAPL_20160115_Put,AAPL_20160122_Call,AAPL_20160122_Put,AAPL_20160129_Call,AAPL_20160129_Put,AAPL_20160205_Call,AAPL_20160205_Put,...,AAPL_20191018_Call,AAPL_20191018_Put,AAPL_20200117_Call,AAPL_20200117_Put,AAPL_20200619_Call,AAPL_20200619_Put,AAPL_20210115_Call,AAPL_20210115_Put,AAPL_20210618_Call,AAPL_20210618_Put
2019-02-13,0,0,0,0,0,0,0,0,0,0,...,0,0,3804,4383,750,916,2465,2812,115,302
2019-02-14,0,0,0,0,0,0,0,0,0,0,...,0,0,5962,5895,801,160,591,560,207,36
2019-02-15,0,0,0,0,0,0,0,0,0,0,...,0,0,4380,4052,1004,191,712,201,337,467
2019-02-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-02-19,0,0,0,0,0,0,0,0,0,0,...,544,950,6416,3387,1668,1793,1238,433,265,979
2019-02-20,0,0,0,0,0,0,0,0,0,0,...,528,4626,3792,5746,324,1278,2317,1896,578,625
2019-02-21,0,0,0,0,0,0,0,0,0,0,...,183,1133,3337,7488,444,205,1057,911,298,330


In [9]:
df6 = obj.get_volume(expdates='02/25/19') # raise Error if expdates not exist

ValueError: No None data expired at 02/25/19 was found, formatting option volume failed

------------------------
## OptionVolume_20190215

In [10]:
import pandas as pd
from OptionVolume_20190215 import OptionVolume # import OptionVolume in file OptionVolume_20190215.py
file_location = '../data/template_20190215.xlsx'
workbook = pd.ExcelFile(file_location)

In [11]:
obj = OptionVolume(workbook,'AAPL')
df1 = obj.get_volume(expdates=['02/15/19','02/22/19']) # return daily volume that aggregated on all call/put strikes for 2 expiration dates
df2 = obj.get_volume(expdates=['02/15/19','02/22/19'],call_put='Call') # return call option daily volume that aggregated on all strikes for 2 expiration dates
df3 = obj.get_volume(expdates='02/22/19',call_put='P') # return put option daily volume that aggregated on all strikes for 02/22/19
df4 = obj.get_volume(call_put = 'Put') # return put option daily volume that aggregated on all strikes for all existed expiration dates

In [12]:
obj.expdates_existed # all expiration dates existed in data

['04/18/19',
 '01/17/20',
 '06/18/21',
 '03/01/19',
 '02/15/19',
 '01/15/21',
 '03/08/19',
 '02/22/19',
 '06/19/20',
 '06/21/19',
 '07/19/19',
 '03/15/19',
 '03/22/19',
 '03/29/19']

In [13]:
df1.iloc[-5:]

,AAPL_02/15/19,AAPL_02/22/19
Dates,,
2019-02-11,197570.0,23460.0
2019-02-12,130102.0,32205.0
2019-02-13,191659.0,41509.0
2019-02-14,177168.0,53513.0
2019-02-15,190436.0,83582.0


In [14]:
df2.iloc[-5:]

,AAPL_02/15/19_Call,AAPL_02/22/19_Call
Dates,,
2019-02-11,84601.0,16236.0
2019-02-12,84327.0,23076.0
2019-02-13,89176.0,27858.0
2019-02-14,108604.0,34046.0
2019-02-15,109342.0,51014.0


In [15]:
df3.iloc[-5:]

,AAPL_02/22/19_P
Dates,
2019-02-11,7224.0
2019-02-12,9129.0
2019-02-13,13651.0
2019-02-14,19467.0
2019-02-15,32568.0


In [16]:
df4.iloc[-5:]

,AAPL_04/18/19_Put,AAPL_01/17/20_Put,AAPL_06/18/21_Put,AAPL_03/01/19_Put,AAPL_02/15/19_Put,AAPL_01/15/21_Put,AAPL_03/08/19_Put,AAPL_02/22/19_Put,AAPL_06/19/20_Put,AAPL_06/21/19_Put,AAPL_07/19/19_Put,AAPL_03/15/19_Put,AAPL_03/22/19_Put,AAPL_03/29/19_Put
Dates,,,,,,,,,,,,,,
2019-02-11,22116.0,3107.0,1672.0,2291.0,112969.0,1056.0,2718.0,7224.0,226.0,3084.0,797.0,30366.0,1009.0,532.0
2019-02-12,4158.0,1888.0,91.0,2305.0,45775.0,267.0,775.0,9129.0,1135.0,8310.0,2036.0,21192.0,413.0,748.0
2019-02-13,20441.0,4359.0,300.0,2268.0,102483.0,2810.0,995.0,13651.0,913.0,7404.0,1203.0,22212.0,570.0,324.0
2019-02-14,3847.0,5895.0,36.0,2794.0,68564.0,556.0,970.0,19467.0,155.0,6602.0,4707.0,11901.0,962.0,600.0
2019-02-15,8485.0,4032.0,461.0,6703.0,81094.0,198.0,1231.0,32568.0,185.0,2658.0,621.0,15935.0,1371.0,780.0


In [17]:
df5 = obj.get_volume(expdates='02/25/19') # raise Error if expdates not exist

ValueError: No  data expired at 02/25/19 was found, formatting option volume failed